## Import necessary modules
Run this cell before running any other cells

In [55]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

ModuleNotFoundError: No module named 'bleach'

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [5]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

2025-01-30 16:31:51,499 | INFO     |: info
2025-01-30 16:31:51,500 | WARNING  |: warning
2025-01-30 16:31:51,501 | ERROR    |: error
2025-01-30 16:31:51,501 | CRITICAL |: critical


<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [49]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

NameError: name 'get_ble_controller' is not defined

## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [4]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

13.5


In [5]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

[->9.000<-]


## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [6]:
ble.send_command(CMD.PING, "")

In [7]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

PONG


The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [8]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

## Disconnect

In [9]:
# Disconnect
ble.disconnect()

2025-01-29 10:50:21,338 | INFO     |: Disconnected from C0:C3:E4:25:23:51


In [39]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2025-01-30 17:26:48,999 | INFO     |: Already connected to a BLE device


In [ ]:
ECHO

In [21]:
ble.send_command(CMD.ECHO, "HiHello")
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

Robot says -> HiHello :)
2025-01-29 11:17:38,029 | INFO     |: Disconnected from C0:C3:E4:25:23:51


In [ ]:
SEND_THREE_FLOATS

In [28]:
ble.send_command(CMD.SEND_THREE_FLOATS, "0.1|22.2|4.3")

2025-01-29 11:32:52,115 | INFO     |: Disconnected from C0:C3:E4:25:23:51


In [ ]:
GET_TIME_MILLIS 

In [78]:
ble.send_command(CMD.GET_TIME_MILLIS , "")

Received msg : 665708
Received msg : 665708
Received msg : 665708
Received msg : 665708
Received msg : 665708


In [ ]:
Notification Handler

In [51]:
msg = None
def notif_handler(uuid,byte_array):
    global msg
    received_msg = ble.bytearray_to_string(byte_array)
    msg = received_msg[3:]
    print(f"Received msg : {str(msg)}")
ble.start_notify(ble.uuid['RX_STRING'], notif_handler)


In [79]:
ble.send_command(CMD.GET_TIME_MILLIS , "")

Received msg : 675769
Received msg : 675769
Received msg : 675769
Received msg : 675769
Received msg : 675769
2025-01-29 12:03:30,021 | INFO     |: Disconnected from C0:C3:E4:25:23:51


In [ ]:
Loop

In [13]:
msg = []
ble.send_command(CMD.GET_TIME_MILLIS_1s , "")

In [ ]:
print("number of message received within 1 second: ", len(msg) )
print("Effective transmittion rate: ", len(msg), "msg/sec")

In [ ]:
send_time_data

In [52]:
received_msg = []
ble.send_command(CMD.TIME_DATA_ARRAY, "20|50")

ble.send_command(CMD.SEND_TIME_DATA, "20")



Received msg : 6821
Received msg : 6871
Received msg : 6921
Received msg : 6971
Received msg : 7021
Received msg : 7071
Received msg : 7121
Received msg : 7171
Received msg : 7221
Received msg : 7271
Received msg : 7321
Received msg : 7371
Received msg : 7421
Received msg : 7471
Received msg : 7521
Received msg : 7571
Received msg : 7621
Received msg : 7671
Received msg : 7721
Received msg : 7771


In [56]:
received_msg = []
ble.send_command(CMD.TIME_TEMP_DATA_ARRAY, "20|50")

ble.send_command(CMD.GET_TEMP_READINGS, "20")


Received msg : e:93311    Temp:77
Received msg : e:93362    Temp:76
Received msg : e:93412    Temp:76
Received msg : e:93462    Temp:77
Received msg : e:93512    Temp:77
Received msg : e:93562    Temp:77
Received msg : e:93612    Temp:77
Received msg : e:93662    Temp:75
Received msg : e:93712    Temp:77
Received msg : e:93762    Temp:76
Received msg : e:93812    Temp:76
Received msg : e:93862    Temp:77
Received msg : e:93912    Temp:77
Received msg : e:93962    Temp:77
Received msg : e:94012    Temp:77
Received msg : e:94062    Temp:76
Received msg : e:94112    Temp:78
Received msg : e:94162    Temp:77
Received msg : e:94212    Temp:76
Received msg : e:94262    Temp:77
